# SPCS Networking Connectivity Test: Microsoft Sharepoint

**Note: This Notebook should be run in an SPCS Container for testing to be valid**

1. Please provide your Sharepoint URL
2. Check that the Network Rule structure matches your requirements
3. Run the Connectivity Test if your EAI is already configured
4. Use the Cells to prepare the EAI if needed, then rerun the test.

In [ ]:
# SharePoint Connectivity Test Configuration
# Update the SHAREPOINT_SITE_URL with your actual SharePoint site URL

# This role will be used to create the EAI and other objects if necessary
IMPLEMENTATION_ROLE = "ACCOUNTADMIN"

# This role will be used by Openflow to run Connectors and perform your Data Engineering tasks
OPENFLOW_RUNTIME_ROLE = "OPENFLOWRUNTIMEROLE"

# Site URL is necessary for creating the EAI, and if you want to run the Connectivity test
SHAREPOINT_SITE_URL = "https://abc.sharepoint.com/sites/xyz"  # Replace with your actual SharePoint URL

# For Connectivity test and discovery
SHAREPOINT_CLIENT_ID = "REPLACEME"
SHAREPOINT_CLIENT_SECRET = "REPLACEME"
SHAREPOINT_TENANT_ID = "REPLACEME"


MICROSOFT_LOGIN_URL = "https://login.microsoft.com"
MICROSOFT_GRAPH_URL = "https://graph.microsoft.com"

print(f"Configuration:")
print(f"SharePoint Site: {SHAREPOINT_SITE_URL}")
print("\nReady to test connectivity...")

In [ ]:
import requests
from urllib.parse import urlparse

def test_connectivity(url, description):
    """Test HTTP connectivity to a given URL"""
    try:
        print(f"\n🔍 Testing {description}: {url}")

        # Simple HTTP test - let EAI handle the routing
        response = requests.get(url, timeout=10, allow_redirects=True)
        print(f"  ✅ HTTP Response: {response.status_code}")

        # Any response means connectivity is working
        return True

    except requests.exceptions.ConnectionError as e:
        print(f"  ❌ Connection Error: {e}")
        return False
    except requests.exceptions.Timeout as e:
        print(f"  ❌ Timeout Error: {e}")
        return False
    except Exception as e:
        print(f"  ❌ Error: {e}")
        return False

# Run connectivity tests
print("=" * 60)
print("SHAREPOINT CONNECTIVITY TEST")
print("=" * 60)

results = {}
results['sharepoint'] = test_connectivity(SHAREPOINT_SITE_URL, "SharePoint Site")
results['login'] = test_connectivity(MICROSOFT_LOGIN_URL, "Microsoft Login")
results['graph'] = test_connectivity(MICROSOFT_GRAPH_URL, "Microsoft Graph API")

print("\n" + "=" * 60)
print("CONNECTIVITY TEST SUMMARY")
print("=" * 60)

all_passed = True
for service, passed in results.items():
    status = "✅ PASS" if passed else "❌ FAIL"
    print(f"{service.upper()}: {status}")
    if not passed:
        all_passed = False

if all_passed:
    print(f"\n🎉 All connectivity tests PASSED!")
    print("Your SPCS environment can reach all required SharePoint endpoints.")
    print("You can proceed with SharePoint integration.")
else:
    print(f"\n⚠️  Some connectivity tests FAILED!")
    print("You need to configure Snowflake External Access Integration (EAI).")
    print("Please run the EAI setup cells below to resolve connectivity issues.")

## SharePoint Connector Configuration Discovery

**Important**: The SharePoint UI can display localized or translated names, but the API returns the actual names. For example, the UI might show "Documents" while the API returns Japanese characters or a completely different name.

This cell will help you discover the EXACT values that the SharePoint API returns, which you must use when configuring the OpenFlow connector. Do not trust what you see in the SharePoint UI - use the values shown below instead.


In [ ]:
import requests
import json
from urllib.parse import urlparse

def get_access_token(tenant_id, client_id, client_secret):
    """Retrieve OAuth2 access token for Microsoft Graph API"""
    token_url = f"https://login.microsoftonline.com/{tenant_id}/oauth2/v2.0/token"
    data = {
        'grant_type': 'client_credentials',
        'client_id': client_id,
        'client_secret': client_secret,
        'scope': 'https://graph.microsoft.com/.default'
    }
    try:
        response = requests.post(token_url, data=data, timeout=10)
        response.raise_for_status()
        return response.json().get('access_token')
    except Exception as e:
        print(f"ERROR: Failed to obtain access token")
        print(f"Details: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"Response: {e.response.text}")
        return None

def get_site_id(access_token, site_url):
    """Retrieve SharePoint Site ID from site URL"""
    parsed = urlparse(site_url)
    hostname = parsed.netloc
    site_path = parsed.path
    api_url = f"https://graph.microsoft.com/v1.0/sites/{hostname}:{site_path}"
    headers = {'Authorization': f'Bearer {access_token}', 'Accept': 'application/json'}
    try:
        response = requests.get(api_url, headers=headers, timeout=10)
        response.raise_for_status()
        return response.json().get('id')
    except Exception as e:
        print(f"ERROR: Failed to obtain Site ID")
        print(f"Details: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"Response: {e.response.text}")
        return None

def get_drives(access_token, site_id):
    """Retrieve all drives (document libraries) for a site"""
    api_url = f"https://graph.microsoft.com/v1.0/sites/{site_id}/drives"
    headers = {'Authorization': f'Bearer {access_token}', 'Accept': 'application/json'}
    try:
        response = requests.get(api_url, headers=headers, timeout=10)
        response.raise_for_status()
        return response.json().get('value', [])
    except Exception as e:
        print(f"ERROR: Failed to retrieve drives")
        print(f"Details: {e}")
        if hasattr(e, 'response') and e.response:
            print(f"Response: {e.response.text}")
        return []

def list_drive_root_items(access_token, drive_id, max_items=5):
    """List first few items in drive root to verify access"""
    api_url = f"https://graph.microsoft.com/v1.0/drives/{drive_id}/root/children"
    headers = {'Authorization': f'Bearer {access_token}', 'Accept': 'application/json'}
    try:
        response = requests.get(api_url, headers=headers, timeout=10)
        response.raise_for_status()
        items = response.json().get('value', [])
        return items[:max_items]
    except Exception as e:
        print(f"    WARNING: Could not list drive contents - {e}")
        return []

# Execute discovery
print("=" * 80)
print("SHAREPOINT CONNECTOR CONFIGURATION VALUES")
print("=" * 80)
print("\nAuthenticating with Microsoft Graph API...")

access_token = get_access_token(SHAREPOINT_TENANT_ID, SHAREPOINT_CLIENT_ID, SHAREPOINT_CLIENT_SECRET)

if not access_token:
    print("\nFailed to authenticate. Check your TENANT_ID, CLIENT_ID, and CLIENT_SECRET.")
else:
    print("Successfully authenticated")

    print("\nRetrieving Site ID...")
    site_id = get_site_id(access_token, SHAREPOINT_SITE_URL)

    if not site_id:
        print("\nFailed to get Site ID. Check your Site URL and app permissions.")
    else:
        print(f"Site ID: {site_id}")

        print("\nDiscovering Document Libraries (Drives)...")
        drives = get_drives(access_token, site_id)

        if not drives:
            print("No drives found or access denied.")
        else:
            print(f"\nFound {len(drives)} document library/libraries:")
            print("\n" + "=" * 80)

            for idx, drive in enumerate(drives, 1):
                drive_name = drive.get('name', 'N/A')
                drive_id = drive.get('id', 'N/A')
                web_url = drive.get('webUrl', 'N/A')

                print(f"\nLibrary #{idx}:")
                print(f"  API Name (USE THIS): '{drive_name}'")
                print(f"  Drive ID: {drive_id}")
                print(f"  Web URL: {web_url}")

                # List some files to verify access
                items = list_drive_root_items(access_token, drive_id)
                if items:
                    print(f"  Sample files/folders ({len(items)} shown):")
                    for item in items:
                        item_type = "Folder" if item.get('folder') else "File"
                        print(f"    - [{item_type}] {item.get('name')}")
                else:
                    print(f"  (Empty or cannot list contents)")
                print("-" * 80)

            # Clear summary for connector configuration
            print("\n" + "=" * 80)
            print("CONNECTOR CONFIGURATION PARAMETERS")
            print("=" * 80)
            print(f"\nSite URL: {SHAREPOINT_SITE_URL}")
            print(f"Site ID: {site_id}")
            print(f"\nDocument Library Names (copy EXACTLY as shown):")
            for drive in drives:
                drive_name = drive.get('name')
                print(f"  '{drive_name}'")

            print("\n" + "=" * 80)
            print("IMPORTANT: Use the EXACT library names shown above in your connector.")
            print("These may differ from what you see in the SharePoint UI!")
            print("=" * 80)


## External Access Integration (EAI) Setup

If Connectivity testing fails, you can use the cells below to prepare and implement an EAI suitable for SharePoint access.

After creating and setting the EAI, return to Cell 2 to run the connectivity test again, then run the API discovery cells to validate your credentials and discover configuration IDs.

In [ ]:
-- Create Network Rules for SharePoint connectivity
-- Run this cell if connectivity tests failed

USE ROLE {{IMPLEMENTATION_ROLE}};

-- Network rule for Microsoft login services
CREATE OR REPLACE NETWORK RULE sharepoint_services_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('login.microsoftonline.com:443', 'login.microsoft.com:443', 'www.office.com:443','graph.microsoft.com:443', 'developer.microsoft.com:443');

-- Network rule for SharePoint sites (modify domain as needed)
-- You may need to update this with your specific SharePoint domain
CREATE OR REPLACE NETWORK RULE sharepoint_site_rule
  MODE = EGRESS
  TYPE = HOST_PORT
  VALUE_LIST = ('*.sharepoint.com:443');

SHOW NETWORK RULES LIKE 'sharepoint_%';

In [ ]:
-- Create External Access Integration for SharePoint
-- This combines all the network rules into a single integration

CREATE OR REPLACE EXTERNAL ACCESS INTEGRATION sharepoint_eai
  ALLOWED_NETWORK_RULES = (
    sharepoint_services_rule,
    sharepoint_site_rule
  )
  ENABLED = TRUE
  COMMENT = 'External Access Integration for Microsoft SharePoint connectivity';

-- Grant usage on the integration to your roles
GRANT USAGE ON INTEGRATION sharepoint_eai TO ROLE {{IMPLEMENTATION_ROLE}};
GRANT USAGE ON INTEGRATION sharepoint_eai TO ROLE {{OPENFLOW_RUNTIME_ROLE}};

SHOW EXTERNAL ACCESS INTEGRATIONS LIKE 'sharepoint_eai';

In [ ]:
ALTER NOTEBOOK EAI_SHAREPOINT
  SET EXTERNAL_ACCESS_INTEGRATIONS = ('SHAREPOINT_EAI');

After Creating and Setting the EAI on the Notebook, run the Connectivity test again.